In [25]:
from langchain.prompts import  PromptTemplate
from langchain.tools import DuckDuckGoSearchRun
from langchain.agents import Tool
from langchain.agents import AgentType
from langchain.agents import initialize_agent
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatGooglePalm
from langchain import LLMChain
from langchain.llms import GooglePalm
import re
from key import google_api_key
from datetime import datetime

pattern = r'[^\w\s]'
    

search = DuckDuckGoSearchRun()
tools = [
    Tool(
        name = "Current Search",
        func=search.run,
        description="useful for when you need to answer questions about current events or the current state of the world"
    ),
]



In [26]:
class careerNinjaModel:
    def __init__(self,open_api_key = None):
        #Model declaration to chat with the user

        self.username = None
        self.llm_chat = ChatGooglePalm(google_api_key = google_api_key)
     
        self.memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
        self.careerNinjaModel_llm  = initialize_agent(tools,
                               self.llm_chat,
                               agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
                                verbose=False,
                            handle_parsing_errors=True,
                              memory=self.memory)

        #Model declaration to check the tone of the sentences
        self.prompt_template  = PromptTemplate(
                input_variables=['topic'],
                template="Please analyze the tone of content related to '{topic}' and classify it as either friendly or non-friendly.no additional information" )
        self.check_tone_model  =LLMChain(prompt=self.prompt_template,llm=GooglePalm(google_api_key = google_api_key))
        self.sentence = None
        self.user_tone = None

        
        #Model declaration to collect user information
        self.data_prompt_template = PromptTemplate(
                input_variables=['user_deatils'],
                template="Ask the user politely and in a innovative way about there {user_deatils} in English only one question in first person" )
        self.user_data_model  =LLMChain(prompt=self.data_prompt_template,llm=ChatGooglePalm(google_api_key = google_api_key))
        self.verification = None
        

    def check_tone(self):
        try:
            predection = self.check_tone_model.run(topic = self.sentence)
            if predection == "friendly":
                self.user_tone = True
            else:
                self.user_tone = False      
        except Exception as e:
             self.user_tone = False

    
    def user_promt(self):
        ans = self.careerNinjaModel_llm(self.sentence)
        if self.username != None:
            return "Hello " +self.username + "Information is" + ans['output']
        return ans

    
    def ask_user(self,detail):
    
        question = self.user_data_model.run(user_deatils = detail)
        return question

    
    def check_user_details(self,text,key):
        data_prompt_template = PromptTemplate(
                    input_variables=['key','text'],
                    template="Is this {text} valid {key} just give the answer as yes or no" )
        
        user_model_check = LLMChain(prompt=data_prompt_template,llm=GooglePalm(google_api_key = google_api_key))
        try:
            predection = user_model_check.run(key = key,text = text)
            if predection == "yes":
                self.verification = True
            else:
                self.verification = False
        except Exception as e:
            self.verification = False

    from datetime import datetime

    def validate_birthday(self,birthday):
        try:
            birthdate = datetime.strptime(birthday, '%Y-%m-%d')
            

            current_date = datetime.now()
            
            age = current_date.year - birthdate.year - ((current_date.month, current_date.day) < (birthdate.month, birthdate.day))
            if age < 150 and birthdate <= current_date:
                return True,
            else:
                return False
        except ValueError:
            return False
            
    def validate_phone_number(self,phone_number):

            pattern = r'^\d{10}$'
            if re.match(pattern, phone_number):
                return True
            else:
                return False
                
    def validate_email(self,email):
 
            pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
            if re.match(pattern, email):
                return True
            else:
                return False
            
        
        
model = careerNinjaModel()      

In [ ]:
user_data = {"name":None,"birthday":None,"Address":None,"Phone-Number":None,"Email":None}

print("Hi How are you what would you like to know about ???")
while True:
    if user_data['name']!=None:
                    model.username = user_data['name']
    user_input = str(input())
    model.sentence = user_input
    ans = model.user_promt()
    print("----------------------------------------------------------------------------------------------------------------------")
    print("Chatbot response : ", ans)
    print("----------------------------------------------------------------------------------------------------------------------")
    model.check_tone()
    if model.user_tone:
        for key, value in user_data.items():
            if value is None:
     
                question = model.ask_user(key)
                result_string = re.sub(pattern, '', question)
                print("----------------------------------------------------------------------------------------------------------------------")
                print("Chatbot response:",result_string)
                print("----------------------------------------------------------------------------------------------------------------------")
                user_response = input()
                
                if key ==  'name':
                    print("----------------------------------------------------------------------------------------------------------------------")
                    print("Chatbot response: Nice to mee you",user_response)
                    print("----------------------------------------------------------------------------------------------------------------------")
                if key == 'birthday':
                    predection = model.validate_birthday(user_response)
                    if predection:
                        user_data['birthday'] = user_response
                    else:
                        print("----------------------------------------------------------------------------------------------------------------------")
                        print("Chatbot response : No problem lets continue with our Topic today")
                        print("----------------------------------------------------------------------------------------------------------------------")
                        break
                elif key == 'address':
                if validate_address(user_response):
                    user_data['address'] = user_response
                else:
                        print("----------------------------------------------------------------------------------------------------------------------")
                        print("Chatbot response : No problem lets continue with our Topic today")
                        print("----------------------------------------------------------------------------------------------------------------------")
                        break
                elif key == 'name':
                    if validate_name(user_response):
                        user_data['name'] = user_response
                    else:
                        print("----------------------------------------------------------------------------------------------------------------------")
                        print("Chatbot response : No problem lets continue with our Topic today")
                        print("----------------------------------------------------------------------------------------------------------------------")
                        break
                elif key == 'email':
                    if validate_email(user_response):
                        user_data['email'] = user_response
                    else:
                        print("----------------------------------------------------------------------------------------------------------------------")
                        print("Chatbot response : No problem lets continue with our Topic today")
                        print("----------------------------------------------------------------------------------------------------------------------")
                        break
                elif key == 'phone_number':
                    if validate_phone_number(user_response):
                        user_data['phone_number'] = user_response
                    else:
                        print("----------------------------------------------------------------------------------------------------------------------")
                        print("Chatbot response : No problem lets continue with our Topic today")
                        print("----------------------------------------------------------------------------------------------------------------------")
                        break
                    
                
                

Hi How are you what would you like to know about ???


 hiii how are youuu


----------------------------------------------------------------------------------------------------------------------
Chatbot response :  Hello sahilInformation isAnime is a style of animation popular in Japan. It is characterized by its distinctive art style, which often features large eyes, vibrant colors, and fantastical settings. Anime can be found in a variety of genres, including action, adventure, comedy, drama, romance, and science fiction. It is enjoyed by people of all ages, and has become a popular form of entertainment around the world.

You can watch anime online on Anime-Planet. Anime-Planet is a website that offers a wide variety of anime shows and movies. You can watch anime for free on Anime-Planet, but you will need to create an account. Once you have created an account, you can watch anime online or download it to your computer.

Here are some of the most popular anime shows and movies that you can watch on Anime-Planet:

* Naruto
* One Piece
* Dragon Ball Z
* Bleac

 sahil


----------------------------------------------------------------------------------------------------------------------
Chatbot response: Nice to mee you sahil
----------------------------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------------------------
Chatbot response: Hi there Im just curious when is your birthday Id love to wish you a happy birthday on the day itself
----------------------------------------------------------------------------------------------------------------------


...

----------------------------------------------------------------------------------------------------------------------
Chatbot response: Hi there Im a virtual assistant that can help you with a variety of tasks In order to better assist you may I have your phone number I promise I wont spam you or sell your information
----------------------------------------------------------------------------------------------------------------------


 9980670768


----------------------------------------------------------------------------------------------------------------------
Chatbot response : No problem lets continue with our Topic today
----------------------------------------------------------------------------------------------------------------------


 anime


----------------------------------------------------------------------------------------------------------------------
Chatbot response :  Hello sahilInformation isAnime is a style of animation popular in Japan. It is characterized by its distinctive art style, which often features large eyes, vibrant colors, and fantastical settings. Anime can be found in a variety of genres, including action, adventure, comedy, drama, romance, and science fiction. It is enjoyed by people of all ages, and has become a popular form of entertainment around the world.

You can watch anime online on Anime-Planet. Anime-Planet is a website that offers a wide variety of anime shows and movies. You can watch anime for free on Anime-Planet, but you will need to create an account. Once you have created an account, you can watch anime online or download it to your computer.

Here are some of the most popular anime shows and movies that you can watch on Anime-Planet:

* Naruto
* One Piece
* Dragon Ball Z
* Bleac

In [ ]:
user_data

In [5]:
import pandas as pd

# Sample dictionary


# Create a DataFrame from the dictionary
df = pd.DataFrame(user_data)

# Specify the CSV file name
csv_file = 'data.csv'

# Save the DataFrame to a CSV file
df.to_csv(csv_file, index=False)

print(f'Data has been successfully written to {csv_file}')


ValueError: If using all scalar values, you must pass an index

In [ ]:
LLMChain.generate("To help us celebrate your special day ")

In [6]:
user_data

{'name': 'sahil',
 'birthday': None,
 'Address': None,
 'Phone-Number': None,
 'Email': None}

In [7]:
df

NameError: name 'df' is not defined